## TD2. Vision Transformers
By Nicolas Dufour

In this TD, we will implement the Transformers architecture. Transformers has been a key architecture in deep learning for the past 5 years.

It has first began with NLP, then came audio and finally, since 2020, computer vision.
We will implement every block that makes a transformer from scratch and we will try to create a deep understanding of what is happening.
Here is a figure for the transformer architecture:


<img src="https://www.researchgate.net/profile/Miruna-Gheata/publication/355339249/figure/fig1/AS:1079476452622337@1634378650979/Encoder-decoder-architecture-of-the-Transformer-developed-by-Vaswani-et-al-28.ppm" width=768>

## Instructions
In `pytorch` you must avoid using for loops at all cost. It's almost always possible to find a vectorized version of the operation you want to implement.

**In this TP, the only for-loop you can do is the training loop.**

In [1]:
!pip install einops
!pip install timm

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
import math
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from einops import rearrange, repeat
from PIL import Image
from torchvision import transforms
import requests

torch.manual_seed(0)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Part 1. The attention mechanism

### Implementing the scaled dot-product attention mechanism

The transformer architecture is built around one key block: The attention.
The idea behind attention is the following. Imagine you want to retrieve information from a dictionary. The dictionnary is indexed by keys which maps to a particular value. Now, you have a query which will be matched against the keys of the dict and if you have a match, you will retrieve the associated value.
Attention is very similar to this simple retrieval example. Now, with real data, we don't have this structure, we however are going to learn to create it.

We have 2 sets of vectors (also named tokens). One is $X_{to}$ which is the destination set. We want to be able to map this set of tokens to queries. We achieve this by doing a linear projection of $X_{to}$ to obatain:  $Q = W_QX_{to}$

The other set is $X_{from}$ the set from which we want to retrieve information. We will need to extract both keys and values from this set. We therefore do 2 linear projections of $X_{from}$ to obtain:  $K = W_KX_{from}$ and $V = W_VX_{from}$.

Now, contrary to the dictionnary where queries and values are exact matchs, we don't have this here. Therefore, we will perform a softer match by computing the similarity matrix between $Q$ and $K$. Then for each $Q$, we want to output the values that have the higher similarity. We therefore output the weighted sum of the values, weighted by the softmax of the similarity (also called the attention matrix).

Finally, the attention operation is given by the cross attention:

$$
A(Q,K,V) = \text{SoftMax}(\frac{QK^T}{\sqrt{d_k}})V
$$

We divide the similarity by $\sqrt{d_k}$ for stability reason to avoid the similarity to explode with big vectors which would lead to very sharp attention coeficients.

#### Question 1.
Implement the attention operation, use  `torch.einsum` to easily compute the similarity matrix.

In [3]:
# Xfrom is sequence from encoder output, which has to go through 2 layers (independently), to produce K and V which are used for cross attention in decoder
# Xto is decoder input sequence, projected through a linear layer to produce Q

class Attention(nn.Module):
    def __init__(self, x_to_dim, x_from_dim, hidden_dim,):
        # To complete
        super(Attention, self).__init__()
        self.x_to_proj = nn.Linear(x_to_dim, hidden_dim)
        self.x_from_proj_key = nn.Linear(x_from_dim, hidden_dim)
        self.x_from_proj_val = nn.Linear(x_from_dim, hidden_dim)
        self.scale = hidden_dim ** 0.5



    def forward(self, x_to, x_from):
        # x_to = [batch size, x_to_len, x_to_dim]
        # x_from = [batch size, x_from_len, x_from_dim]

        # To complete
        Q = self.x_to_proj(x_to)  # [batch size, x_to_len, hidden_dim]
        K = self.x_from_proj_key(x_from)  # [batch size, x_from_len, hidden_dim]
        V = self.x_from_proj_val(x_from)  # [batch size, x_from_len, hidden_dim]

        # similarity matrix using einsum
        similarity = torch.einsum('bih,bjh->bij', Q, K)
        similarity = similarity / self.scale

        # attention weights
        attention = F.softmax(similarity, dim=-1)

        #apply attention weights to V
        x_to = torch.einsum('bij,bjh->bih', attention, V)

        return x_to

### Multi-head attention

We improve the above attention implementation by introducing multi-head attention. The idea here is that we compute the attention on subspaces of the $Q,K,V$ triplets.
We split each vector in $n$ subsets and compute the attention for each subset. At the end, we concatenate every attention output and project it with an output projection.

#### Question 2.
Implement Multihead attention.

In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, x_to_dim, x_from_dim, hidden_dim, n_heads):
        # To complete
        super(MultiHeadAttention, self).__init__()
        assert hidden_dim % n_heads == 0, "hidden_dim must be divisible by n_heads"

        self.n_heads = n_heads
        self.head_dim = hidden_dim // n_heads
        self.scale = self.head_dim ** 0.5
        self.hidden_dim = hidden_dim

        self.linear_q = nn.Linear(x_to_dim, hidden_dim)
        self.linear_k = nn.Linear(x_from_dim, hidden_dim)
        self.linear_v = nn.Linear(x_from_dim, hidden_dim)
        self.output_linear = nn.Linear(hidden_dim, hidden_dim)



    def forward(self, x_to, x_from):
        # x_to = [batch size, x_to_len, x_to_dim]
        # x_from = [batch size, x_from_len, x_from_dim]

        # To complete

        #batch size is the number of samples processed in parallel,
        #x to len is the length of each sequence (number of tokens), and
        #x to dim is the dimensionality of each token representation

        batch_size, x_to_len, _ = x_to.shape
        _, x_from_len, _ = x_from.shape

        Q = self.linear_q(x_to)  # [batch_size, x_to_len, hidden_dim]
        K = self.linear_k(x_from)  # [batch_size, x_from_len, hidden_dim]
        V = self.linear_v(x_from)  # [batch_size, x_from_len, hidden_dim]

        #####
        # reshape Q K V for multi-head attention
        Q = Q.view(batch_size, x_to_len, self.n_heads, self.head_dim).transpose(1, 2)  # [batch_size, n_heads, x_to_len, head_dim]
        K = K.view(batch_size, x_from_len, self.n_heads, self.head_dim).transpose(1, 2)  # [batch_size, n_heads, x_from_len, head_dim]
        V = V.view(batch_size, x_from_len, self.n_heads, self.head_dim).transpose(1, 2)  # [batch_size, n_heads, x_from_len, head_dim]

        attention_scores = torch.einsum('bnqd,bnkd->bnqk', Q, K) / self.scale

        # softmax to get attention weights
        attention_weights = F.softmax(attention_scores, dim=-1)

        # attention weights to values using einsum
        out = torch.einsum('bnqk,bnvd->bnqd', attention_weights, V)

        #concat heads and apply output projection
        out = out.transpose(1, 2).contiguous().view(batch_size,x_to_len,self.hidden_dim)
        output = self.output_linear(out)
        ######
        return output

MultiheadAttention is the attention that is used in transformers in pratice. It is used in 2 flavors:
- Self Attention: When $X_{to}$ attends itself ($X_{to}=X_{from}$)
- Cross Attention. $X_{to}\neq X_{from}$


#### Question 3.
Implement MultiHead Self Attention and MultiHeadCrossAttention

In [6]:
class MultiHeadSelfAttention(MultiHeadAttention):
    def __init__(self, x_dim, hidden_dim, n_heads):
        # To complete
        super(MultiHeadSelfAttention, self).__init__(x_to_dim=x_dim, x_from_dim=x_dim,
                                                     hidden_dim=hidden_dim, n_heads=n_heads)

    def forward(self, x):
        # x = [batch size, x_len, x_dim]
        # To complete
        return super(MultiHeadSelfAttention, self).forward(x_to=x, x_from=x)

class MultiHeadCrossAttention(MultiHeadAttention):
    def __init__(self, x_to_dim, x_from_dim, hidden_dim, n_heads):
        # To complete
        super(MultiHeadCrossAttention, self).__init__(x_to_dim=x_to_dim, x_from_dim=x_from_dim,
                                                     hidden_dim=hidden_dim, n_heads=n_heads)

    def forward(self, x_to, x_from):
        # x_to = [batch size, x_to_len, x_to_dim]
        # x_from = [batch size, x_from_len, x_from_dim]

        # To complete
        return super(MultiHeadCrossAttention, self).forward(x_to=x_to, x_from=x_from)

### LayerNorm
Another key component of the transformer is the LayerNorm. As we have previously seen, normalizing the output of a deep learning layer helps a lot with convergence and stability.
Until Transformers, the most used normalization is BatchNorm. We normalize the data among the batch dimension. However, this has a few problems.
- The normalization depend on the other samples in the batch
- When using multiple GPUs, BatchNorm needs to synchronize the batch statistic across GPUs, which locks the forward process and slow down training.

The last element is the most important one. Transformers, aims to be a easy to parralilize architecture and can't afford to use batchnorm.

Instead, Transformers uses Layer Norm. LayerNorm is sample dependent, which removes the synchronization issue. We normalize over the channel dimension instead of the batch dimension.

<img src="https://production-media.paperswithcode.com/methods/Screen_Shot_2020-05-19_at_4.24.42_PM.png">

To account for the loss of capacity, we map the output by a linear transformation with a learned bias and scale.

#### Question 4.
Implement the LayerNorm

In [7]:
class LayerNorm(nn.Module):
    # To complete
    def __init__(self, parameters_shape, dim_to_norm=-1, eps=1e-5):
        super(LayerNorm, self).__init__()
        self.parameters_shape = parameters_shape
        self.eps = eps
        self.dim_to_norm = dim_to_norm

        #learnable parameters
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta = nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, x):
        mean = x.mean(dim=self.dim_to_norm, keepdim=True)
        var = x.var(dim=self.dim_to_norm, keepdim=True, unbiased=False)

        x_normalized = (x - mean) / torch.sqrt(var + self.eps)

        return self.gamma * x_normalized + self.beta



### Feed Feedward Network

Finally, the last block is a feed-forward network with one hidden layer. This layer has usually a size of $2 * input\_dim$. This is followed by a dropout layer and an activation function. Here, we will use leaky relu, with a leak parameter of 0.1.

#### Question 5.
Implement the FFN layer

In [8]:
class FFN(nn.Sequential):
    def __init__(self, hidden_dim, dropout_rate=0.1, expansion_factor=2):
        # To complete
        super(FFN, self).__init__(
            nn.Linear(hidden_dim, hidden_dim * expansion_factor),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim * expansion_factor, hidden_dim)
        )

### The Transformer block

The last thing that we are missing are the skip connection. Like in ResNet, the transformer architecture implements the skip-connection. This allow for a better gradient flow avoiding vanishing gradient.
There is a skip connection after the attention and the feed forward network

#### Question 6.
Given at the transformer figure at the top, implement the Transformer Encoder Block

In [9]:
class TransformerEncoderBlock(nn.Module):
    def __init__(self ,data_dim, hidden_dim, n_heads, dropout_rate=0.1):
        # To complete
        super(TransformerEncoderBlock, self).__init__()


        self.multi_head_attention = MultiHeadSelfAttention(data_dim, hidden_dim, n_heads)
        self.layer_norm1 = LayerNorm(hidden_dim)
        self.ffn = FFN(hidden_dim, dropout_rate=dropout_rate)
        self.layer_norm2 = LayerNorm(hidden_dim)


    def forward(self, x):
        # x = [batch size, x_len, hidden dim]
        # To complete
        multi_head_attention_output = self.multi_head_attention(x)
        x = self.layer_norm1(x + multi_head_attention_output)

        ffn_output = self.ffn(x)
        x = self.layer_norm2(x + ffn_output)
        return x


### Positional embedding
The transformers architecture is permutation independent. That means that for every token, we can swap 2 tokens and have the exact same result. However, the position of the token can be a very important information to consider. Imagine in an image. If a pixel is nearby another pixel, we want the transformer to be able to capture such information. Which is not the case for now.
That's why we introduce positional encodings. For each token, add the positional encoding to the original token:

$$
X_i = X_i + PE(i)
$$

with X_i the token at the i dimension.

The most used positional encodings are sinusoidal encodings. They are defined as follow:

$$
PE(i, 2j) = sin(i / 10000^{\frac{2j}{d}}) \\
PE(i, 2j + 1) = cos(i / 10000^{\frac{2j}{d}})
$$


Where $d$ the dimension of the tokens, $i$, the i-th token in the sequence and $2j$ (resp $2j + 1$), the index of the dimension of the vector.
The idea here is that we add a sinusoidal that encode the position in a multidimensional array.

Another common positional encodings is the learned positional encoding. Simply, we let the network learn a set of tensor $PE$ that match the sequence length and dimension of the tokens.

#### Question 7.

Implement both Sinusoidal and Learned positional embeddings

In [10]:
class SinusoidalPositionalEncoding(nn.Module):
    def __init__(self, hidden_dim):
        super(SinusoidalPositionalEncoding, self).__init__()
        self.hidden_dim = hidden_dim

    def forward(self, x):
        # x = [batch size, seq len, hidden dim]
        batch_size, seq_len, _ = x.shape
        # Tensor for positional encodings, no batch dimension needed
        PE_mat = torch.zeros(seq_len, self.hidden_dim)

        position = torch.arange(seq_len).unsqueeze(1)  # [seq_len, 1]

        # div_term calculation adjusted for the entire hidden_dim
        #log for numerical stability and exp to adhere back to formula
        div_term = torch.exp(torch.arange(0, self.hidden_dim, 2).float() * -(torch.log(torch.tensor(10000.0)) / (self.hidden_dim // 2)))

        # sin to even indices
        PE_mat[:, 0::2] = torch.sin(position * div_term)  # [seq_len, hidden_dim/2]

        # cos to odd indices
        PE_mat[:, 1::2] = torch.cos(position * div_term)  # [seq_len, hidden_dim/2]

        # expand to match batch size
        PE_mat = PE_mat.unsqueeze(0).expand(batch_size, -1, -1)  # [batch_size, seq_len, hidden_dim]

        return x + PE_mat

class LearnedPositionalEncoding(nn.Module):
    def __init__(self, hidden_dim, max_len):
        # To complete
        super(LearnedPositionalEncoding, self).__init__()
        self.hidden_dim = hidden_dim
        self.max_len = max_len
        #learnable param
        self.PE = nn.Parameter(torch.randn(1, max_len, hidden_dim))


    def forward(self, x):
        # x = [batch size, seq len, hidden dim]
        # To complete
        batch_size, seq_len, _ = x.shape
        PE_mat = self.PE[:, :seq_len, :]
        # match batch size
        PE_mat = PE_mat.expand(batch_size, -1, -1)
        return x + PE_mat

### The transformer encoder
Now you have everything you need to implement the transformer . You add positional encoding to the tokens and then stack N transformer encoder layers

#### Question 8.
Implement the transformer encoder with n_layers and the ability to choose both positional embeddings.

Tip: Look into `ModuleList`

In [11]:
class TransformerEncoder(nn.Module):
    def __init__(self, data_dim,  hidden_dim, n_heads, n_layers, dropout_rate=0.1, positional_encoding="sinusoidal", max_len=1000):
        # To complete
        super(TransformerEncoder, self).__init__()
        self.positional_encoding = SinusoidalPositionalEncoding(hidden_dim) if positional_encoding == "sinusoidal" else LearnedPositionalEncoding(hidden_dim, max_len)
        self.layers = nn.ModuleList([TransformerEncoderBlock(data_dim, hidden_dim, n_heads, dropout_rate) for _ in range(n_layers)])

    def forward(self, x):
        # x = [batch size, seq len, hidden dim]
        # To complete
        x = self.positional_encoding(x)
        for layer in self.layers:
            x = layer(x)
        return x

# Part 2. Classical Architectures: ViT & CCT

## The Vision Transformer
The above architecture was introduced in 2017 to process sequences of text tokens. However, it could be useful to be able to leverage this architecture for computer vision. On the contrary of convolutional neural network, the transformer has the advantage to introduce less inductive bias.

This could be interesting to leverage to improve vision systems. If we learn the biases from the data, we can hope to have better performances. We however need compute and a lot of data to do this.

To apply the transformer to images, one key question remains to be answered: How do we transform an image to tokens? The approach introduce in Vision Transformers is to cut the image into patches that are then transformed into a token trhought a linear projection.

We also add an extra token, known as the classification token, that will be the token which will be use to predict upon. After going through the N transformer layers, this is the token that goes throught a multi layer perceptron.


<img src= "https://1.bp.blogspot.com/-_mnVfmzvJWc/X8gMzhZ7SkI/AAAAAAAAG24/8gW2AHEoqUQrBwOqjhYB37A7OOjNyKuNgCLcBGAsYHQ/s1600/image1.gif" width="512">


#### Question 9

Implement the vision transformer

Hint: Use Conv2D with the right kernel size and stride to do the linear projection of non-overlapping patches.

In [12]:
class ViT(nn.Module):
    def __init__(self, patch_size, hidden_dim, n_heads, n_layers, n_classes, dropout_rate=0.1, positional_encoding="sinusoidal", max_len=1000, num_classes=10):
        super(ViT, self).__init__()
        # To complete

        self.patch_size = patch_size
        self.patch_embedding = nn.Conv2d(in_channels=3, out_channels=hidden_dim,
                                         kernel_size=patch_size, stride=patch_size)
        self.transformer_encoder = TransformerEncoder(hidden_dim, hidden_dim, n_heads, n_layers, dropout_rate, positional_encoding, max_len)
        self.classification_token = nn.Parameter(torch.randn(1, 1, hidden_dim))

        self.mlp_head = nn.Linear(hidden_dim, n_classes)


    def forward(self, x):
        # x = [batch size, 3, image height, image width]
        batch_size, _, height, width = x.shape
        # To complete

        # seperate each image into patches
        x = self.patch_embedding(x)  # [batch_size, hidden_dim, height/patch_size, width/patch_size]
        x = x.flatten(2).transpose(1, 2)  #[batch_size, num_patches, hidden_dim]

        # add classification token to the sequence
        cls_tokens = self.classification_token.expand(batch_size, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        # pass through transformer encoder
        x = self.transformer_encoder(x)

        # classification token goes through the final MLP
        cls_token = x[:, 0, :]
        x = self.mlp_head(cls_token)

        return x


#### Question 10.
Train a ViT on CIFAR10 for 100 epochs (for compute reason you can use only 20 epochs) and log both train and test loss and accuracy.
We provide a data augmentation strategy called auto augment to avoid overfitting on the training data.
Hparameters are to be choosen to your discretion.


Tips for Hparams:
- Don't use a transformer hidden dim too big (<256)
- Use a small patch size
- Use AdamW with some weight decay to avoid overfitting
- Use between 2 and 6 transformer layers.
- Use between 2 and 4 transformer heads

In [ ]:
batch_size = 128
train_set = CIFAR10(root='./data', train=True, download=True, transform=transforms.Compose([
    transforms.autoaugment.AutoAugment(policy=transforms.AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
]))

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)

test_set = CIFAR10(root='./data', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
]))

test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:
# To complete: train the model, (don't forget to test it)
# hyperparameters
patch_size = 4
hidden_dim = 128
n_heads = 4
n_layers = 4
dropout_rate = 0.1
epochs = 20
learning_rate = 0.001
weight_decay = 0.01  # for AdamW

model = ViT(patch_size=patch_size, hidden_dim=hidden_dim, n_heads=n_heads, n_layers=n_layers, n_classes=n_classes, dropout_rate=dropout_rate).to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)


for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    # training metrics
    train_loss /= len(train_loader)
    train_acc = 100. * correct / total

    # testing / validation
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    # testing metrics
    test_loss /= len(test_loader)
    test_acc = 100. * correct / total

    print(f'Epoch [{epoch+1}/{epochs}]')
    print(f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%')
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.2f}%')





############ TODO######

# ACTUALLY RUN THIS, COULD NOT FOR NOW BECAUSE CANNOT USE GPU NOT ENOUGH CREDITS :(




In [ ]:
# Plot the training and test loss

## Compact Convolutional Transformer
The previous network requires a lot of compute and data to be trained. As we mentionned before, the transformer removes the inductive bias of convnets which requires more data to be trained.

To bypass this, let's try out another architecture. We will try an hybrid architecture that preserves the inductive biases of convolution but manages to use the transformer to add global learning.

The first change is the tokenizer. We replace it with a ConvNet. Each convnet layer has a convolution, ReLU and maxpooling.
The second change is to actually remove the classfication token and classify on top of a pooling of all tokens. The pooling is done with an attention like mechanism:
- For each sample, we predict a scalar, that we compute the softmax over all the sample tokens.
- We then do an weighted average pool by this softmax values over the tokens. The weight is given by the previous step

More details see: https://arxiv.org/abs/2104.05704

<img src= https://miro.medium.com/v2/resize:fit:720/format:webp/1*8diH01Fl7MhHRemLy9hUHw.png width=512>

#### Question 11.
Implement the Convolutional based tokenizer and the SeqPool operationm

In [ ]:
class ConvPatchEmbedding(nn.Module):
    def __init__(self, n_layers, kernel_size, hidden_dim):
        # To complete
        pass

    def forward(self, x):
        # x = [batch size, 3, image height, image width]
        # To complete
        pass

class SeqPool(nn.Module):
    def __init__(self, hidden_dim):
        # To complete
        pass

    def forward(self, x):
        # x = [batch size, seq len, hidden dim]
        # To complete
        pass

#### Question 12.

Implement the Compact Convolutional Transformer.

In [ ]:
class CCT(nn.Module):
    def __init__(self, n_conv_layers, kernel_size,  n_transformer_layers, hidden_dim, n_heads, n_classes, dropout_rate=0.1):
        # To complete
        pass

    def forward(self, x):
        # x = [batch size, 3, image height, image width]
        # To complete
        pass

#### Question 13.
Train the CCT on CIFAR-10 for 100 epochs (for compute reason you can use only 20 epochs) and log both train and test loss and accuracy. You should obtain at least 75+% test accuracy, and observe a improvement compared to the previous ViT (Possible to get 90%+).
We provide a data augmentation strategy called auto augment to avoid overfitting on the training data.
Hparameters are to be choosen to your discretion.

Tips for Hparams:
- Don't use too big of a transformer hidden dim (<256)
- For the convnet, aim to have between 32 and 128 output tokens.
- Use AdamW with some weight decay to avoid overfitting
- Use between 2 and 6 transformer layers.
- Use between 2 and 4 transformer heads

Training takes around 30min (depending of hparams).

In [ ]:
batch_size = 128
train_set = CIFAR10(root='./data', train=True, download=True, transform=transforms.Compose([
    transforms.autoaugment.AutoAugment(policy=transforms.AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
]))

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)

test_set = CIFAR10(root='./data', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
]))

test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=4)


In [ ]:
# To complete: train the model, (don't forget to test it)

In [ ]:
# Plot the training and test loss